In [1]:
import numpy as np

In [2]:
%%time
x = np.random.rand(512*2**17)
y = np.random.rand(512*2**17)
for _ in range(100):
    z = x + y

CPU times: user 10.1 s, sys: 3.47 s, total: 13.6 s
Wall time: 13.6 s


In [3]:
13.6/100

0.136

In [6]:
(512/0.136)*3/1024

11.029411764705882

In [11]:
import nlcpy as vp

In [12]:
x = vp.random.rand(1024*2**17)
y = vp.random.rand(1024*2**17)

In [16]:
%%time
with vp.prof.ftrace_region('sum'):
    for _ in range(100):
        z = x + y

CPU times: user 510 ms, sys: 3.6 ms, total: 513 ms
Wall time: 511 ms


In [2]:
511/1000/100

0.00511

In [5]:
1024/0.00511*3/1024

587.0841487279844

In [17]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
!/opt/nec/ve/bin/ftrace

*----------------------*
  FTRACE ANALYSIS LIST
*----------------------*

Execution Date : Thu Jul  7 19:19:29 2022 JST
Total CPU Time : 0:00'11"686 (11.686 sec.)


FREQUENCY  EXCLUSIVE       AVER.TIME     MOPS   MFLOPS  V.OP  AVER.    VECTOR L1CACHE CPU PORT VLD LLC PROC.NAME  
           TIME[sec](  % )    [msec]                    RATIO V.LEN      TIME    MISS     CONF HIT E.%            

     2720    11.100( 95.0)     4.081  15613.9   3869.4  99.13 256.0    11.089   0.011    0.000    0.05 nlcpy_add_f64_f64_f64
      340     1.235( 10.6)     3.633  17538.0   4346.2  99.13 256.0     1.233   0.002    0.000    0.04  -thread0
      340     1.422( 12.2)     4.181  15238.9   3776.5  99.13 256.0     1.420   0.001    0.000    0.05  -thread1
      340     1.422( 12.2)     4.181  15238.7   3776.4  99.13 256.0     1.420   0.001    0.000    0.05  -thread2
      340     1.432( 12.3)     4.212  15126.1   3748.5  99.13 256.0     1.431   0.001    0.000    0.04  -thread3
      340     1.383( 11.8) 

In [11]:
import numpy as np
import nlcpy as vp

In [12]:
%%time
x = vp.random.rand(2**27)
y = vp.random.rand(2**27)
z = vp.empty_like(x)

CPU times: user 38.3 ms, sys: 468 µs, total: 38.8 ms
Wall time: 38.7 ms


In [13]:
z.size*8

1073741824

In [14]:
import nlcpy as vp
ve_lib = vp.jit.CustomVELibrary(
    code="""
        int ve_sum(double *px, double *py, double *pz, int n) {
            #pragma omp parallel for
            for (int i = 0; i  < n; i++) pz[i] = px[i] + py[i];
            return 0;
        }
"""
, cflags=(*vp.jit.get_default_cflags(), "-ftrace"), ldflags=(*vp.jit.get_default_ldflags(), "-ftrace"))

In [15]:
ve_lib

<CustomVELibrary(
* code:

        int ve_sum(double *px, double *py, double *pz, int n) {
            #pragma omp parallel for
            for (int i = 0; i  < n; i++) pz[i] = px[i] + py[i];
            return 0;
        }

* path: None
* cflags:  -c -fpic -O2 -I /opt/anaconda3/lib/python3.8/site-packages/nlcpy/include -fopenmp -ftrace
* ldflags:  -fpic -shared -fopenmp -ftrace
* log_stream: None
* compiler: /opt/nec/ve/bin/ncc
* use_nlc: False
* ftrace: False
* ID: 1892634_2022-07-07.18:11:43.965039
* src_path: /tmp/tmpzpi20ie3/1892634_2022-07-07.18:11:43.965039.c
* obj_path: /tmp/tmpzpi20ie3/1892634_2022-07-07.18:11:43.965039.o
* lib_path: /tmp/tmpzpi20ie3/1892634_2022-07-07.18:11:43.965039.so
* lib: <nlcpy.veo._veo.VeoLibrary object at 0x7fae7fd1fd60>
)>

In [16]:
import nlcpy.ve_types as ve_types

In [17]:
ve_sum = ve_lib.get_function(
    "ve_sum",
    args_type=(ve_types.uint64, ve_types.uint64, ve_types.uint64, ve_types.int32),
    ret_type=ve_types.int32
)

In [18]:
%%time
ve_sum(x.ve_adr, y.ve_adr, z.ve_adr, z.size)

CPU times: user 26 µs, sys: 21 µs, total: 47 µs
Wall time: 50.3 µs


In [19]:
%%time
with vp.prof.ftrace_region('ve_sum'):
    for _ in range(100):
        ve_sum(x.ve_adr, y.ve_adr, z.ve_adr, z.size)

CPU times: user 460 ms, sys: 2.94 ms, total: 463 ms
Wall time: 462 ms


In [20]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
!/opt/nec/ve/bin/ftrace

*----------------------*
  FTRACE ANALYSIS LIST
*----------------------*

Execution Date : Thu Jul  7 18:11:56 2022 JST
Total CPU Time : 0:00'03"722 (3.722 sec.)


FREQUENCY  EXCLUSIVE       AVER.TIME     MOPS   MFLOPS  V.OP  AVER.    VECTOR L1CACHE CPU PORT VLD LLC PROC.NAME  
           TIME[sec](  % )    [msec]                    RATIO V.LEN      TIME    MISS     CONF HIT E.%            

      808     3.686( 99.0)     4.561  14846.7   3678.2  99.10 256.0     3.519   0.004    0.000    0.08 ve_sum$1
      101     0.462( 12.4)     4.578  14806.0   3664.6  99.00 256.0     0.380   0.000    0.000    0.06  -thread0
      101     0.461( 12.4)     4.561  14847.5   3678.4  99.10 256.0     0.440   0.000    0.000    0.08  -thread1
      101     0.461( 12.4)     4.560  14851.5   3679.4  99.10 256.0     0.440   0.000    0.000    0.08  -thread2
      101     0.460( 12.4)     4.557  14855.0   3681.4  99.13 256.0     0.460   0.000    0.000    0.06  -thread3
      101     0.460( 12.4)     4.558  148

In [9]:
462/1000/100

0.00462

In [10]:
1./_

216.45021645021646

In [11]:
_*3

649.3506493506494